In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220527.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Washington Nationals,Colorado Rockies,0.516414,0.483586,-116.0,-102.0,0.537037,0.504950,-0.020623,-0.021364,0.000000,0.000000,0.000000,0.000000
1,Boston Red Sox,Baltimore Orioles,0.621097,0.378903,-180.0,152.0,0.642857,0.396825,-0.021760,-0.017922,0.000000,0.000000,0.000000,0.000000
2,Detroit Tigers,Cleveland Guardians,0.335453,0.664547,134.0,-158.0,0.427350,0.612403,-0.091898,0.052144,0.000000,0.013453,0.000000,1381.092192
3,New York Mets,Philadelphia Phillies,0.602061,0.397939,-158.0,134.0,0.612403,0.427350,-0.010342,-0.029412,0.000000,0.000000,0.000000,0.000000
4,Tampa Bay Rays,New York Yankees,0.497977,0.502023,-126.0,108.0,0.557522,0.480769,-0.059545,0.021254,0.000000,0.004093,0.000000,420.212817
5,Atlanta Braves,Miami Marlins,0.604712,0.395288,-148.0,126.0,0.596774,0.442478,0.007938,-0.047190,0.001969,0.000000,202.098861,0.000000
6,Minnesota Twins,Kansas City Royals,0.650174,0.349826,-184.0,154.0,0.647887,0.393701,0.002287,-0.043875,0.000649,0.000000,66.667892,0.000000
7,St. Louis Cardinals,Milwaukee Brewers,0.441175,0.558825,122.0,-144.0,0.450450,0.590164,-0.009275,-0.031339,0.000000,0.000000,0.000000,0.000000
8,Arizona Diamondbacks,Los Angeles Dodgers,0.345170,0.654830,132.0,-156.0,0.431034,0.609375,-0.085864,0.045455,0.000000,0.011636,0.000000,1194.586015
9,San Diego Padres,Pittsburgh Pirates,0.718206,0.281794,-220.0,184.0,0.687500,0.352113,0.030706,-0.070318,0.009826,0.000000,1008.710154,0.000000


In [4]:
bets.to_csv('past_bets/base/bets_20220527.csv')

In [5]:
# # Instantating constants

# regex_weight = r"\d{3}\sl"
# regex_reach = r'\d{2}"'
# regex_dob = r',\s\d{4}'
# regex_various_stats = r'\d{1}\.\d{1,2}'
# regex_various_stats_2 = r'\d{1,2}%'
# regex_record = r'\d{1,2}-\d{1,2}-\d{1,2}'
# year = dt.date.today().year
# fight_data = pd.DataFrame()
# options = Options()
# options.add_argument('--no-sandbox')
# options.add_argument("user-data-dir=/Users/hsinger24/Library/Application Support/Google/Chrome/Default1")
# options.add_argument("--start-maximized")
# options.add_argument('--disable-web-security')
# options.add_argument('--allow-running-insecure-content')
# options.add_argument("--disable-setuid-sandbox")
# driver = webdriver.Chrome(ChromeDriverManager().install())
# driver.get('http://ufcstats.com/statistics/events/completed')
upcoming_card_data = pd.DataFrame(columns = ['name', 'weight', 'reach', 'age', 'slpm', 'sapm', 'td_avg', 'sub_avg', 'strk_acc', 'strk_def', 'td_acc','td_def', 'wins', 'losses'])

# # Getting data for upcoming card

# links_home_page = driver.find_elements(By.TAG_NAME, 'a') 
# del links_home_page[:6] 
# upcoming_card = links_home_page[0] 
# upcoming_card.click() 
# time.sleep(2)
# links_upcoming_card = driver.find_elements(By.TAG_NAME, 'a') 
# del links_upcoming_card[:4]
# for i in range(7):
#     del links_upcoming_card[-1]
# for i, link in enumerate(links_upcoming_card):
#     if link.text=='View\nMatchup':
#         del links_upcoming_card[i]
# num_fighters = len(links_upcoming_card)
# fighter = links_upcoming_card[0]
# fighter.click()
# for i in range(num_fighters):
#     if i!=0:
#         links_upcoming_card = driver.find_elements(By.TAG_NAME, 'a')
#         del links_upcoming_card[:4]
#         for j, link in enumerate(links_upcoming_card):
#             if link.text=='View\nMatchup':
#                 del links_upcoming_card[j]
#         fighter= links_upcoming_card[i]
#         fighter.click() 
#     time.sleep(2)
#     html = driver.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     name = soup.find_all('span', class_ = 'b-content__title-highlight')[0].text.replace('\n', '').replace(' ', '')
#     weight = float(re.findall(regex_weight, soup.prettify())[0].strip('l').replace(' ', ''))
#     reach = float(re.findall(regex_reach, soup.prettify())[1].strip('"'))
#     dob = int(re.findall(regex_dob, soup.prettify())[0].strip(',').replace(' ', ''))
#     age = year - dob
#     slpm = float(re.findall(regex_various_stats, soup.prettify())[1])
#     sapm = float(re.findall(regex_various_stats, soup.prettify())[2])
#     td_avg = float(re.findall(regex_various_stats, soup.prettify())[3])
#     sub_avg = float(re.findall(regex_various_stats, soup.prettify())[4])
#     strk_acc = float(re.findall(regex_various_stats_2, soup.prettify())[0].strip('%'))
#     strk_def = float(re.findall(regex_various_stats_2, soup.prettify())[1].strip('%'))
#     td_acc = float(re.findall(regex_various_stats_2, soup.prettify())[2].strip('%'))
#     td_def = float(re.findall(regex_various_stats_2, soup.prettify())[3].strip('%'))
#     record = re.findall(regex_record, soup.prettify())[0]
#     record = record.split('-')
#     wins = float(record[0])
#     losses = float(record[1])
#     list_of_stats = [name, weight, reach, age, slpm, sapm, td_avg, sub_avg, strk_acc, strk_def, td_acc, td_def, wins, losses]
#     series = pd.Series(list_of_stats, index = upcoming_card_data.columns)
#     upcoming_card_data = upcoming_card_data.append(series, ignore_index = True)
#     driver.back()
# evens = upcoming_card_data.iloc[::2]
# evens.reset_index(drop = True, inplace = True)
# odds = upcoming_card_data.iloc[1::2]
# odds.reset_index(drop = True, inplace = True)
# final= pd.merge(evens, odds, left_on = evens.index, right_on = odds.index)
# final.drop('key_0', axis = 1, inplace = True)
# final.columns = ['fighter_1', 'weight_1', 'reach_1', 'age_1', 'slpm_1', 'sapm_1', 'td_avg_1', 'sub_avg_1', 
#                 'strk_acc_1', 'strk_def_1', 'td_acc_1','td_def_1', 'wins_1', 'losses_1', 'fighter_2', 'weight_2', 
#                 'reach_2', 'age_2', 'slpm_2', 'sapm_2', 'td_avg_2', 'sub_avg_2', 'strk_acc_2', 'strk_def_2', 
#                 'td_acc_2','td_def_2', 'wins_2', 'losses_2']
# final['result'] = 0
# print(final.tail())
# driver.quit()

AttributeError: type object 'object' has no attribute 'dtype'